In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import Image

import os, sys, re, datetime, time
from pathlib import Path

pj_dir = Path(os.getcwd()).parents[0]
data_dir = pj_dir/'data'
img_dir = pj_dir/'images'
src_dir = pj_dir/'src'
sys.path.append(str(src_dir))

from matplotlib import pyplot as plt
import japanize_matplotlib
import seaborn as sns
plt.style.use("bmh")
import numpy as np
import pandas as pd
import dask.dataframe as dd

from tqdm import tqdm_notebook
from dotenv import load_dotenv

In [2]:
from requests_html import HTML

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [4]:
options = Options()
options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--single-process')
options.add_argument('--disable-dev-shm-usage')

In [5]:
driver = webdriver.Chrome(chrome_options=options)

/Users/yutaro_ikeda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [6]:
endpoint = 'https://www-shibuya.jp/schedule/#wwwxwww/'

In [7]:
ls_events = []

In [8]:
from dateutil.relativedelta import relativedelta

In [9]:
today = datetime.datetime.today()

In [10]:
from requests_html import HTML

In [11]:
driver.get(endpoint)
time.sleep(3)

for i in range(3): # 3ヶ月分
    current_date = today + relativedelta(month=i)
    year = current_date.year
    month = current_date.month
    
    html = HTML(html=driver.page_source, url='')
    articles = html.find('#eventList > div > article')
    for article in articles:        
        day = int(article.find('.date .day', first=True).text)
        datetime_ = datetime.date(year, month, day)

        url = article.find('.pageLink', first=True).attrs.get('href')
        place = article.find('.inner .placeLabel span', first=True).text
        artist = article.find('.inner .info .title', first=True).text
        title = article.find('.inner .info .exp span', first=True).text
        openstart = article.find('.inner .info .openstart', first=True).text
        oss = openstart.replace('OPEN / START\u3000', '').split('/')
        if len(oss) == 2:
            open_, start = oss

        dic_event = {
            'datetime': datetime_, 'url': url, 'place': place, 'artist': artist, 'title': title,
            'open': open_, 'start': start, 'open_start': oss
        }
        ls_events.append(dic_event)
    next_button = driver.find_element_by_css_selector('#schedule > * > ul > li.next > a')
    next_button.click()

In [12]:
df = pd.DataFrame(ls_events)

In [18]:
load_dotenv(pj_dir/'.env')
os.environ.get('AWS_ACCESS_KEY'), 
from boto3.session import Session

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY')
aws_secret_access_key = os.environ.get('AWS_SECRET_KEY')

session = Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
dynamodb = session.resource('dynamodb', region_name='ap-northeast-1')

In [36]:
table_name = 'live-scheduler-crawler-dev'
table = dynamodb.Table(table_name)

In [59]:
'Hello Wo rld'.lower().replace(' ', '')

'helloworld'

In [60]:
for event in ls_events:
    id_ = "{}_{}".format(event['place'].lower().replace(' ', ''), event['datetime'].strftime('%Y%m%d'))
    item = {
        'id': id_,
        'title': event['title'],
        'date': event['datetime'].strftime('%Y-%m-%d'),
        'place': event['place'], 
        'artist': event['artist'],
        'open': event['open'],
        'start': event['start']
    }

    item = {k: v for k, v in item.items() if v}
    table.put_item(Item=item)

In [33]:
from boto3.dynamodb.conditions import Key, Attr

In [47]:
table.scan(
    FilterExpression=Attr('date').between('2018-12-18', '2018-12-20')
)

{'Items': [{'artist': 'HAMIDASYSTEM',
   'date': '2018-12-18',
   'place': 'WWW',
   'open': '18:15 ',
   'id': 'WWW_20181218',
   'start': ' 19:00',
   'title': 'HAMIDASYSTEM2周年記念ワンマンライブ「黎明の貴方」'},
  {'artist': 'MOROHA',
   'date': '2018-12-18',
   'place': 'WWW X',
   'open': '18:30 ',
   'id': 'WWW X_20181218',
   'start': ' 19:30',
   'title': 'MOROHA BEST RELEASE TOUR 〜置いていかれた曲達〜'},
  {'artist': 'FIZZY POP(ナチュラル炭酸)',
   'date': '2018-12-19',
   'place': 'WWW X',
   'open': '18:00 ',
   'id': 'WWW X_20181219',
   'start': ' 19:00',
   'title': 'FIZZYPOP 2nd ONEMAN LIVE 〜Take Off to the SKY〜 Day1 - Black Night -'},
  {'artist': 'AAAMYYY / STEPHENSMITH',
   'date': '2018-12-19',
   'place': 'WWW',
   'open': '19:30 ',
   'id': 'WWW_20181219',
   'start': ' 20:00',
   'title': 'WWW presents dots'},
  {'artist': '4年2組 / 他',
   'date': '2018-12-20',
   'place': 'WWW',
   'open': '17:30 ',
   'id': 'WWW_20181220',
   'start': ' 18:30',
   'title': '4年2組単独クラス会@渋谷WWW 〜今年の嫌なこと全部忘れちまえっ！〜'},
 

In [ ]:
'.next .pageLink'

In [50]:
e = ls_events[0]

In [51]:
e.date.strftime('%Y%m%d')

'20181101'

In [40]:
from datetime import datetime

In [1]:
from dotenv import load_dotenv

import os
from pathlib import Path

pj_dir = Path(os.getcwd()).parents[1]

load_dotenv(pj_dir/'.env')

os.environ.get('AWS_ACCESS_KEY'), 

from boto3.session import Session

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY')
aws_secret_access_key = os.environ.get('AWS_SECRET_KEY')

session = Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
dynamodb = session.resource('dynamodb', region_name='ap-northeast-1')

In [21]:
table_name = 'Foo'
table = dynamodb.Table(table_name)

In [22]:
test_data = {
    'id': 'hogehoge', 
    'test': 'hello22',
    'foo': 'bar'
}

table.put_item(Item=test_data)

{'ResponseMetadata': {'RequestId': 'MCK1I1N5NDHSR7INJSD2M90QANVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 25 Nov 2018 06:14:27 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'MCK1I1N5NDHSR7INJSD2M90QANVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}